In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from django.contrib.gis.geos import Point, LineString as DjangoLineString
import pytz

In [2]:
from simple_history.utils import bulk_create_with_history
from helpers.histories import bulk_update_with_history

In [3]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )
#     print(road_set)

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def get_connected_reference(company, resource, key, value):
    value=value.strip()
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [8]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    if field == 'Nome da placa':
        value=value.upper()
    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [9]:
def get_sign(field, sign_field, occ, value):
    if sign_field == 'Cor':
        value=value.capitalize()
    elif sign_field == 'Tipo de Película':
        value=value.split(' ')[1]
    values = occ.form_fields['fields']
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [10]:
!cat .env

STAGE=CCR_PRODUCTION


In [31]:
# filename = 'acessos_importacao_v2'
filename='SH ZB BR-116 SP RJ - Inventário Corrigido' # 16/05/23

In [32]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[0]
# sheetname
ws = wb[sheetname]
ws

<Worksheet "SH ZB BR 116 SP RJ">

In [33]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    if obj['Rodovia'] is not None:           
        obj['img'] = [a for a in ws._images if a.anchor._from.row == index]
 
    values.append(obj)

In [34]:
for a in values:
    if a['Localização'] == 'Alça':
        a['Localização'] = "Alça de Acesso"
    if a['Localização'] == 'Marginal 1':
        a['Localização'] = "Marginal"
    if a['Faixa'] == 'Marginal':
        a['Faixa'] = "Marginal 1"
    if a['Cor_1'].capitalize() == 'Branca':
        a['Cor_1'] = "Branco"
    if a['Cor_2'].capitalize() == 'Branca':
        a['Cor_2'] = "Branco"
    if a['Cor_1'].capitalize() == 'Amarela':
        a['Cor_1'] = "Amarelo"
    if a['Cor_2'].capitalize()== 'Amarela':
        a['Cor_2'] = "Amarelo"

In [35]:
values[34]

{'Identificação ANTT': ' ZE 116 SP 000+190 N 1',
 'km': '000+190',
 'Status': 'Identificado',
 'Equipe': 'Civil: SGS',
 'Encontrado em': datetime.datetime(2022, 11, 22, 0, 0),
 'Executado em': datetime.datetime(2022, 11, 22, 0, 0),
 'Latitude': -22.51558,
 'Longitude': -44.702,
 'Sentido': 'Norte',
 'Classe': 'Sinalização Horizontal (Zebrado)',
 'Faixa': 'Alça',
 'Rodovia': 'BR-116 SP',
 'Localização': 'Alça de Acesso',
 'Observações': None,
 'Tipo_1': 'Linha de Canalização (LCA)',
 'Cor_1': 'Amarelo',
 'Tipo de zebrado_1': 'Até 10 zebras',
 'Tipo_2': 'Zebrado de preenchimento (ZPA)',
 'Cor_2': 'Amarelo',
 'Tipo de zebrado_2': 'Até 10 zebras',
 'img': []}

In [36]:
company = Company.objects.get(name='CCR - Rio Sp')

occurrence_type=OccurrenceType.objects.get(uuid='1b9c3688-1cbf-479a-888b-add5d36b516c')
user=User.objects.get(username='rlcs')
utc = pytz.timezone('America/Sao_Paulo')
company,occurrence_type

(<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>,
 <OccurrenceType: Sinalização Horizontal (Zebrado) - ['CCR - Rio Sp']>)

In [37]:
for a in tqdm(values):
    a['Cores'] = list(filter(None, [{'type': get_sign("Adicionar retrorrefletância de Zebrado","Tipo",occurrence_type,a["Tipo_1"]),'color': get_sign("Adicionar retrorrefletância de Zebrado","Cor",occurrence_type,a["Cor_1"]), 'zebrado_type': get_sign("Adicionar retrorrefletância de Zebrado","Tipo de zebrado",occurrence_type,a["Tipo de zebrado_1"])} if a['Tipo_1'] is not None else None,
                                    {'type': get_sign("Adicionar retrorrefletância de Zebrado","Tipo",occurrence_type,a["Tipo_2"]),'color': get_sign("Adicionar retrorrefletância de Zebrado","Cor",occurrence_type,a["Cor_2"]), 'zebrado_type': get_sign("Adicionar retrorrefletância de Zebrado","Tipo de zebrado",occurrence_type,a["Tipo de zebrado_2"])} if a['Tipo_2'] is not None else None]))

  0%|          | 0/1608 [00:00<?, ?it/s]

In [38]:
for a in values:
    if '-' not in str(a['Latitude']) and a['Latitude'] is not None:
        a['Latitude'] = float("-"+str(a['Latitude']))
    if '-' not in str(a['Longitude']) and a['Longitude'] is not None:
        a['Longitude'] = float("-"+str(a['Longitude']))

In [41]:
objects=[]

for index, a in enumerate(tqdm(values)):

    point, road = get_road_coordinates(a['Rodovia'], float(str(a['km']).replace('+','.')), str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize())) , company)
    if road.name != a['Rodovia']:
        print(road.name,a['Rodovia'])
        
    try:
        objects.append(Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    firm= Firm.objects.get(company=company, name=a['Equipe']),
                    lane=get_connected_reference(company, 'reporting', 'lane', a['Faixa']),
                    road=road,
                    road_name=road.name,
                    direction=get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize()),
                    created_by=user,
                    found_at=a['Encontrado em'],
                    executed_at=a['Executado em'],
                    km=float(str(a['km']).replace('+','.')),
                    end_km_manually_specified= True if 'km final' in a else False,
                    end_km=float(a['km final']) if 'km final' in a and a['km final'] is not None else None,
                    point=point,
                    form_data={
                        "localization": get_value('Localização',occurrence_type,a["Localização"]) if a["Localização"] is not None else None,
                        "id_ccr_antt": a["Identificação ANTT"].strip(),
                        "index": a["Índice (Serial ANTT)"] if a.get('Índice (Serial ANTT)') else None,
                        "is_pedagio": True  if a.get("Zebrado em Pedágio") and a["Zebrado em Pedágio"].lower() == 'sim' else False,
                        "cabine": a["Identificador da Cabine do Pedagio"] if a.get("Identificador da Cabine do Pedagio") else None,
                        "entry_exit_place": get_value('Local (Entrada ou Saída)',occurrence_type,a["Local (Entrada ou Saída)"]) if a.get("Local (Entrada ou Saída)") is not None else None,
                        "retro":a['Cores'] if a['Cores'] is not None else None,
                        "notes": a["Observações"] if a.get('Observações') else None,
                        "latitudedecimal": float(a["Latitude"]) if a.get("Latitude") else None,
                        "longitudedecimal": float(a["Longitude"]) if a.get("Longitude") else None,
                        },
                    ))
    except Exception as e:
        print(f'AQUI: km: {a["km"]} | Rodovia: {a["Rodovia"]} | Linha da Planilha: {index+2}{e}{objecr.form}')
        pass

  0%|          | 0/1608 [00:00<?, ?it/s]

In [42]:
objects[78].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f489c4b7050>,
 'uuid': UUID('af9bb75a-eb7d-4767-8b5d-f9b07de47d1a'),
 'number': '',
 'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
 'road_name': 'BR-116 SP',
 'road_id': 2038,
 'km': 24.98,
 'end_km': None,
 'km_reference': None,
 'project_km': 0,
 'project_end_km': None,
 'point': <Point object at 0x7f486906a1c0>,
 'direction': '1',
 'lane': '29',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': UUID('8e3f277d-cd02-444f-a49b-c8d724f92c9d'),
 'occurrence_type_id': UUID('1b9c3688-1cbf-479a-888b-add5d36b516c'),
 'form_data': {'localization': '1',
  'id_ccr_antt': 'ZE 116 SP 024+980 S 1',
  'index': None,
  'is_pedagio': False,
  'cabine': None,
  'entry_exit_place': None,
  'retro': [{'type': '1', 'color': '2', 'zebrado_type': '2'},
   {'type': '2', 'color': '2', 'zebrado_type': '2'}],
  'notes': None,
  'latitudedecimal': -22.6048,
  'longitudedeci

In [43]:
for reporting in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/1608 [00:00<?, ?it/s]

'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2038: BR-116 SP


/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2022-11-25 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-11-25 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2022-11-25 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: Da

CCR-Inv-2023.76852
'NoneType' object has no attribute 'order'
Searching KM on direction...


/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2022-10-20 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-10-20 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2022-10-20 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: Da

...

CCR-Inv-2023.78241
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78242
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78243
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78244
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78245
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78246
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78247
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on R

CCR-Inv-2023.78300
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78301
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78302
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78303
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78304
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78305
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78306
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78307
'NoneType' object has no attribu

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2022-11-02 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-11-02 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2022-11-02 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: Da

CCR-Inv-2023.78351
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78352
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78353
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78354
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78355
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78356
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78357
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78358
'NoneType' object has no attribu

Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78412
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78413
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78414
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78415
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78416
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78417
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.78418
'NoneType' object has

In [43]:
# for reporting, images in tqdm(objects):
#     for image in reversed(images):
#         reporting_file = ReportingFile(
#             created_by=user,
#             reporting=reporting,
#             description=image.name,
#             km=reporting.km,
#             point=reporting.point
#         )
#         reporting_file.save()
#         image_io = BytesIO(image._data())
#         im = Image.open(image_io)
#         if im.mode in ("RGBA", "P"):
#             im = im.convert("RGB")
#         thumb_io = BytesIO()
#         im.save(thumb_io, format='jpeg', quality=90)
#         reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/585 [00:00<?, ?it/s]

In [20]:
reps=Reporting.objects.filter(company__uuid='3af64f25-59e8-446b-bb1e-963549090b0d',
                              occurrence_type__uuid='1b9c3688-1cbf-479a-888b-add5d36b516c',road_name__startswith='BR-116')
reps.count()

0

In [19]:
# reps.delete()
for a in tqdm(reps[:700]):
    a.delete()

  0%|          | 0/208 [00:00<?, ?it/s]

In [5]:
for a in Reporting.objects.filter(company__uuid='3af64f25-59e8-446b-bb1e-963549090b0d',
                              occurrence_type__uuid='1b9c3688-1cbf-479a-888b-add5d36b516c',road_name__startswith='BR-101'):
    if '-' not in str(a.form_data['longitudedecimal']):
#         a.form_data['longitudedecimal'] = float('-'+str(a.form_data['longitudedecimal']))
        
        print(a.form_data['longitudedecimal'])
    if '-' not in str(a.form_data['latitudedecimal']):
#         a.form_data['latitudedecimal'] = float('-'+str(a.form_data['latitudedecimal']))
        print(a.form_data['latitudedecimal'])
#         a.save()

In [ ]:
teste = get_connected_reference(company, 'reporting', 'lane', value)